<a href="https://colab.research.google.com/github/Ayush-Debnath/tinyLLM/blob/main/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><i><h1>BUILDING A LLM</H1>from sctratch.....</i></b>

<i><H3>Data: The Verdict</h3></i> <br>
loctaion: /content/drive/MyDrive/LLMDATA/the-verdict.txt

<h3><i>Tokenizer</i></h3>

<i>Step 1:</i> Splitting the test in individual words and subword tokens.<br>
<i>Step 2:</i> Convert tokens into token IDs<br>
<i>Step 3:</i> Encode token IDs into vector representations

In [1]:
with open("/content/drive/MyDrive/LLMDATA/the-verdict.txt", "r",encoding="utf-8") as f:
  raw_text=f.read()
print("total number of characters:", len(raw_text))
print(raw_text[:99])


total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re

<h10>Preprocessing

In [3]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:50])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']


In [4]:
print(len(preprocessed))

4690


In [5]:
all_words=sorted(set(preprocessed))
print("vocab size: ",len(all_words))

vocab size:  1130


In [6]:
vocab={token: integer for integer,token in enumerate(all_words)}

In [7]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [8]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}

  def encode(self,text):
    # Reverted the incorrect ']' to ''' and removed the erroneous escape character
    preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)

    preprocessed=[
        item.strip() for item in preprocessed if item.strip()
    ]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    text=re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [9]:
tokenizer=SimpleTokenizerV1(vocab)
text=""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids=tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [10]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

Adding<br>
<|unk|> --> unknown token<br>
<|endoftext|> --> end of text token


In [11]:
all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

In [12]:
vocab={token:integer for integer, token in enumerate(all_tokens)}

In [13]:
len(vocab.items())

1132

In [14]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [32]:
class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}

  def encode(self,text):
    # Fix: Added missing closing parenthesis for the regex pattern
    preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed=[item.strip() for item in preprocessed if item.strip()]
    preprocessed=[
        item if item in self.str_to_int else "<|unk|>" for item in preprocessed
    ]

    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    text=re.sub(r'\s+([,.:;?!"()\'])',r'\1',text)
    return text


In [33]:
tokenizer=SimpleTokenizerV2(vocab)
text1="Hello, do you like some tea?"
text2="In the sunlit terraces of the palace."

text="<|endoftext|> ".join((text1,text2))

In [34]:
text

'Hello, do you like some tea?<|endoftext|> In the sunlit terraces of the palace.'

In [35]:
tokenizer.encode(text)

[1131,
 5,
 355,
 1126,
 628,
 910,
 975,
 10,
 1130,
 55,
 988,
 956,
 984,
 722,
 988,
 1131,
 7]

In [36]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like some tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


<h3><i>BYTE PAIR ENCODING(BPE)</i></h3><br>
It is a subword tokenization algorithm. Most common pair of consecutive bytes of data is replaced witha byte that does not occur in data.

In [37]:
!pip install tiktoken

In [38]:
import importlib
import tiktoken

print("tiktoken version: ",importlib.metadata.version("tiktoken") )

tiktoken version:  0.12.0


In [39]:
tokenizer=tiktoken.get_encoding("gpt2")

In [40]:
text=(
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace"
)

integers=tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271]
